In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None) 

1. Cargad el csv que obtuvimos en los ejercicios de la clase invertida de ayer. 

In [2]:
df_rat = pd.read_csv('Materiales/rating-and-performance-transformado.csv' , index_col=0)

In [3]:
df_rat.head(1)

,title,price,retail_price,units_sold,uses_ad_boosts,rating_x,rating_count,rating_five_count,rating_four_count,rating_three_count,rating_two_count,rating_one_count,badge_local_product,badge_product_quality,badge_fast_shipping,tags,product_color,product_variation_size_id,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,origin_country,merchant_name,merchant_info_subtitle,merchant_rating_count,merchant_id,product_id,listed_products,total_units_sold,mean_units_sold_per_product,rating_y,merchant_ratings_count,mean_product_prices,mean_retail_prices,mean_product_ratings_count,urgency_text_rate,discount_percentage,seller_reputation
0,2020 Summer Vintage Flamingo Print Pajamas Se...,16.0,14,100,No,3.76,54,26.0,8.0,10.0,1.0,9.0,No,Good,No,"Summer,Fashion,womenunderwearsuit,printedpajam...",white,M,Livraison standard,4,No,34,50,CN,zgrdejia,NaN,568,595097d6a26f6e070cb878d1,5e9ae51d43d6a96e303acdb0,1,100,100.0,4.129,568.0,16.0,14.0,54.0,100.0,12.5,Good


2. ¿Qué columnas tienen nulos? 

In [4]:
print("Las columnas que contienen nulos son:\n")

columnas_null = []

for column in df_rat.columns:
    
    if df_rat[column].isna().sum() > 0:
        print(f"{column} --- {df_rat[column].isna().sum()} nulos")
        columnas_null.append(column)

Las columnas que contienen nulos son:

rating_five_count --- 45 nulos
rating_four_count --- 45 nulos
rating_three_count --- 45 nulos
rating_two_count --- 45 nulos
rating_one_count --- 45 nulos
product_color --- 41 nulos
product_variation_size_id --- 14 nulos
origin_country --- 17 nulos
merchant_name --- 4 nulos
merchant_info_subtitle --- 294 nulos
urgency_text_rate --- 668 nulos


3. Explorad los nulos que hay en el DataFrame. 

In [5]:
# Separamos según categóricas o numéricas
cat_columns = [column for column in columnas_null if df_rat[column].dtype == "O"]
num_columns = [column for column in columnas_null if df_rat[column].dtype == "float64"]

In [6]:
# Exploración de los datos categóricos
for column in cat_columns:
    print(f"Revisando {column.upper()}:")
    display(round((df_rat[column].value_counts() / df_rat.shape[0]) * 100,2))
    print(" ")

Revisando PRODUCT_COLOR:


black               19.20
white               16.15
yellow               6.68
pink                 6.29
blue                 6.29
                    ...  
red & blue           0.06
winered & yellow     0.06
claret               0.06
lightpurple          0.06
gold                 0.06
Name: product_color, Length: 101, dtype: float64

 
Revisando PRODUCT_VARIATION_SIZE_ID:


S            40.75
XS           22.63
M            12.71
XXS           6.36
L             3.12
             ...  
White         0.06
32/L          0.06
Size XXS      0.06
Pack of 1     0.06
SIZE S        0.06
Name: product_variation_size_id, Length: 106, dtype: float64

 
Revisando ORIGIN_COUNTRY:


CN    96.38
US     1.97
VE     0.32
SG     0.13
AT     0.06
GB     0.06
Name: origin_country, dtype: float64

 
Revisando MERCHANT_NAME:


广州唯适唯服饰有限公司                              0.95
greatexpectationstechnology              0.76
sjhdstoer                                0.57
shuilingjiaointernationaltradecompany    0.51
sangboostore                             0.51
                                         ... 
asmalltwistofsweetfruit                  0.06
supumawallsticker                        0.06
haoyouduooo                              0.06
successed                                0.06
aishoppingmall                           0.06
Name: merchant_name, Length: 957, dtype: float64

 
Revisando MERCHANT_INFO_SUBTITLE:


86 % avis positifs         8.58
85 % avis positifs         8.26
87 % avis positifs         6.80
88 % avis positifs         6.61
84 % avis positifs         6.48
                           ... 
83% Positives Feedback     0.06
61 % avis positifs         0.06
84% ردود فعل إيجابية       0.06
56 % avis positifs         0.06
84% Positive Feedback      0.06
Name: merchant_info_subtitle, Length: 73, dtype: float64

In [7]:
# Exploración de los valores datos numéricos
for column in num_columns:
    print(f"Revisando {column.upper()}:")
    display(round((df_rat[column].value_counts() / df_rat.shape[0]) * 100,2))
    print(" ")
    
# Exploración de la media y mediana de los datos numéricos
for column in num_columns:
    print(f"Revisando {column.upper()}:")
    desc = df_rat[column].describe()
    print("Media:", desc['mean'])
    print("Mediana:", desc['50%'])
    print(" ")

Revisando RATING_FIVE_COUNT:


1.0       3.12
5.0       3.12
3.0       2.80
2.0       2.73
4.0       2.16
          ... 
786.0     0.06
911.0     0.06
1158.0    0.06
819.0     0.06
258.0     0.06
Name: rating_five_count, Length: 605, dtype: float64

 
Revisando RATING_FOUR_COUNT:


0.0       6.10
1.0       5.79
2.0       4.26
3.0       3.37
4.0       3.37
          ... 
850.0     0.06
1411.0    0.06
239.0     0.06
103.0     0.06
606.0     0.06
Name: rating_four_count, Length: 440, dtype: float64

 
Revisando RATING_THREE_COUNT:


...

 
Revisando RATING_ONE_COUNT:


1.0      7.37
0.0      7.37
3.0      4.83
2.0      4.64
4.0      3.75
         ... 
281.0    0.06
218.0    0.06
52.0     0.06
145.0    0.06
105.0    0.06
Name: rating_one_count, Length: 330, dtype: float64

 
Revisando URGENCY_TEXT_RATE:


50.0     21.49
33.0      9.92
100.0     9.85
25.0      5.85
40.0      4.45
67.0      2.10
20.0      0.95
17.0      0.76
60.0      0.64
63.0      0.51
14.0      0.45
80.0      0.32
75.0      0.25
Name: urgency_text_rate, dtype: float64

 
Revisando RATING_FIVE_COUNT:
Media: 442.2637434554974
Mediana: 79.0
 
Revisando RATING_FOUR_COUNT:
Media: 179.59947643979058
Mediana: 31.5
 
Revisando RATING_THREE_COUNT:
Media: 134.5497382198953
Mediana: 24.0
 
Revisando RATING_TWO_COUNT:
Media: 63.71138743455497
Mediana: 11.0
 
Revisando RATING_ONE_COUNT:
Media: 95.73560209424083
Mediana: 20.0
 
Revisando URGENCY_TEXT_RATE:
Media: 52.226519337016576
Mediana: 50.0
 


4. Con lo aprendido en la lección de hoy gestiona los nulos de la forma que consideres más correcta. Mañana en clase discutiremos sobre los métodos elegidos. 

In [8]:
# Una vez estudiados los datos, se toman las siguientes consideraciones:
# --Reemplazar por 'unknown' aquellas columnas que no tienen una moda definida: product_color, product_variation_size_id, merchant_name, merchant_info_subtitle
# --Reemplazar por la moda aquellas columnas que tienen una variable predominante: origin_country

for column in cat_columns:
    
    if column == 'origin_country':
        
        moda = df_rat[column].mode()[0]
        df_rat[column] = df_rat[column].fillna(moda)
    
    else:
        
        df_rat[column] = df_rat[column].fillna('unknown')

print("Después del reemplazo usando 'fillna' quedan los siguientes nulos")
df_rat[cat_columns].isnull().sum() 

Después del reemplazo usando 'fillna' quedan los siguientes nulos


product_color                0
product_variation_size_id    0
origin_country               0
merchant_name                0
merchant_info_subtitle       0
dtype: int64

In [9]:
# Una vez estudiados los datos, se toman las siguientes consideraciones:
# --Reemplazar por la media aquella columna que no parece tener dispersión de datos: urgency_text_rate
# --Probar KNN Imputer e Iterative Imputer en el resto a ver cuál mantiene datos parecidos

# Quita la columna que reemplazaremos por la media
num_columns.remove('urgency_text_rate')

# Creamos el objeto de la clase del SimpleImputer
imputer = SimpleImputer(strategy = "mean")

df_rat["urgency_text_rate"] = imputer.fit_transform(df_rat[["urgency_text_rate"]])

print(f"Después del 'SimpleImputer' tenemos {df_rat['urgency_text_rate'].isnull().sum()} nulos")

Después del 'SimpleImputer' tenemos 0 nulos


In [10]:
# Creamos dos copias del DF para probar los distintos modos
dfIT = df_rat.copy()
dfKNN = df_rat.copy()

# Probamos a reemplazar con Iterative
imputer_iterative = IterativeImputer(max_iter = 20, random_state = 42)
imputer_iterative_imputado = imputer_iterative.fit_transform(dfIT[num_columns])
dfIT[num_columns] = imputer_iterative_imputado

# Comprobamos los nulos
print(f"Después del 'Iterative' tenemos: \n{dfIT[num_columns].isnull().sum()} nulos")

Después del 'Iterative' tenemos: 
rating_five_count     0
rating_four_count     0
rating_three_count    0
rating_two_count      0
rating_one_count      0
dtype: int64 nulos


In [11]:
# Probamos a reemplazar con KNN Imputer
imputer_knn = KNNImputer(n_neighbors = 5)
imputer_knn_imputado = imputer_knn.fit_transform(dfKNN[num_columns])
dfKNN[num_columns] = imputer_knn_imputado

# Comprobamos los nulos
print(f"Después del 'KNN Imputer' tenemos: \n{dfKNN[num_columns].isnull().sum()} nulos")

Después del 'KNN Imputer' tenemos: 
rating_five_count     0
rating_four_count     0
rating_three_count    0
rating_two_count      0
rating_one_count      0
dtype: int64 nulos


In [14]:
# Exploración de la media y mediana de los datos numéricos

print("REVISIÓN DEL MÉTODO ITERATIVE:\n")
for column in num_columns:
    print(f"Revisando {column.upper()}:")
    desc = dfIT[column].describe()
    print("Media:", desc['mean'])
    print("Mediana:", desc['50%'])
    print(" ")
print("-----------------\n")
print("REVISIÓN DEL MÉTODO KNN IMPUTER:\n")
for column in num_columns:
    print(f"Revisando {column.upper()}:")
    desc = dfKNN[column].describe()
    print("Media:", desc['mean'])
    print("Mediana:", desc['50%'])
    print(" ")
    
# Con ambos métodos hemos obtenido resultados muy parecidos, así que optaremos por aplicar el Iterative dada su rapidez

REVISIÓN DEL MÉTODO ITERATIVE:

Revisando RATING_FIVE_COUNT:
Media: 442.26374345549794
Mediana: 86.0
 
Revisando RATING_FOUR_COUNT:
Media: 179.59947643979058
Mediana: 35.0
 
Revisando RATING_THREE_COUNT:
Media: 134.5497382198951
Mediana: 26.0
 
Revisando RATING_TWO_COUNT:
Media: 63.711387434554915
Mediana: 12.0
 
Revisando RATING_ONE_COUNT:
Media: 95.73560209424078
Mediana: 22.0
 
-----------------

REVISIÓN DEL MÉTODO KNN IMPUTER:

Revisando RATING_FIVE_COUNT:
Media: 442.26374345549794
Mediana: 86.0
 
Revisando RATING_FOUR_COUNT:
Media: 179.59947643979058
Mediana: 35.0
 
Revisando RATING_THREE_COUNT:
Media: 134.5497382198951
Mediana: 26.0
 
Revisando RATING_TWO_COUNT:
Media: 63.711387434554915
Mediana: 12.0
 
Revisando RATING_ONE_COUNT:
Media: 95.73560209424078
Mediana: 22.0
 


In [15]:
# Hacemos el reemplazo definitivo en el DF original
imputer_iterative = IterativeImputer(max_iter = 20, random_state = 42)
imputer_iterative_imputado = imputer_iterative.fit_transform(df_rat[num_columns])
df_rat[num_columns] = imputer_iterative_imputado

# Comprobamos los nulos
print(f"Después del 'Iterative' tenemos: \n{df_rat[num_columns].isnull().sum()} nulos")

Después del 'Iterative' tenemos: 
rating_five_count     0
rating_four_count     0
rating_three_count    0
rating_two_count      0
rating_one_count      0
dtype: int64 nulos


5. Guardad el csv para los ejercicios de la clase invertida de mañana con las columnas sin nulos. 

In [16]:
df_rat.to_csv('Materiales/rating-and-performance-nonull.csv')